In [1]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

You should consider upgrading via the 'c:\users\janbinkowski\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [4]:
# Path to data for train
DATA_PATH = os.path.join('C://Users//JanBinkowski//Desktop//SignLanguageRecognition//MP_DATA')

# Actions to detect
actions = np.array(['a', 'b', 'c'])

# Thirty videos worth of data
no_sequences = 100

# frames for one sequence
sequence_length = 30

In [5]:
sequences, labels = [], []

In [6]:
label_map = {label:num for num, label in enumerate(actions)}

In [7]:
label_map

{'a': 0, 'b': 1, 'c': 2}

In [8]:
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [9]:
X = np.array(sequences)

In [10]:
X_sliced_for_training = np.concatenate((X[:80],X[100:180],X[200:280]), axis=0)

In [11]:
X_sliced_for_validation = np.concatenate((X[80:100],X[180:200],X[280:300]), axis=0)

In [12]:
# X_sliced_for_testing = np.concatenate((X[90:100],X[190:200],X[290:300]), axis=0)

In [13]:
y = to_categorical(labels).astype(int)

In [14]:
y_sliced_for_training = np.concatenate((y[:80],y[100:180],y[200:280]), axis=0)

In [15]:
y_sliced_for_validation = np.concatenate((y[80:100],y[180:200],y[280:300]), axis=0)

In [16]:
# y_sliced_for_testing = np.concatenate((y[90:100],y[190:200],y[290:300]), axis=0)

In [17]:
# X_train, X_test, y_train, y_test = train_test_split(X_sliced_for_training, y_sliced_for_training, test_size=0.05)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [19]:
log_dir = os.path.join('C://Users//JanBinkowski//Desktop//SignLanguageRecognition//Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [20]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [21]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [22]:
# model.fit(X_train, y_train, epochs=200, batch_size=4, validation_data=(X_sliced_for_validation, y_sliced_for_validation), callbacks=[tb_callback])
model.fit(X_train, y_train, epochs=1000, batch_size=4, validation_split=0.2, callbacks=[tb_callback])


Epoch 1/1000
57/57 [==============================] - 5s 50ms/step - loss: 1.0990 - categorical_accuracy: 0.3209 - val_loss: 1.0468 - val_categorical_accuracy: 0.6842
Epoch 2/1000
57/57 [==============================] - 2s 28ms/step - loss: 1.9447 - categorical_accuracy: 0.5940 - val_loss: 1.3393 - val_categorical_accuracy: 0.2456
Epoch 3/1000
57/57 [==============================] - 2s 27ms/step - loss: 1.0807 - categorical_accuracy: 0.3625 - val_loss: 1.0308 - val_categorical_accuracy: 0.2807
Epoch 4/1000
57/57 [==============================] - 1s 25ms/step - loss: 1.1390 - categorical_accuracy: 0.5111 - val_loss: 1.0335 - val_categorical_accuracy: 0.4561
Epoch 5/1000
57/57 [==============================] - 1s 25ms/step - loss: 1.0000 - categorical_accuracy: 0.4845 - val_loss: 1.0603 - val_categorical_accuracy: 0.4035
Epoch 6/1000
57/57 [==============================] - 1s 25ms/step - loss: 1.0101 - categorical_accuracy: 0.5235 - val_loss: 1.4124 - val_categorical_accuracy: 0.386

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            48896     
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 203,459
Trainable params: 203,459
Non-trainable params: 0
__________________________________________________

In [24]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=1)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 30 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions.shape)

Evaluate on test data
15/15 [==============================] - 1s 6ms/step - loss: 1.0985 - categorical_accuracy: 0.3333
test loss, test acc: [1.098517656326294, 0.3333333432674408]
Generate predictions for 30 samples
predictions shape: (15, 3)


In [25]:
model.save('C://Users//JanBinkowski//Desktop//SignLanguageRecognition//Weights//action.h5')